In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Setting the working directory
import os
import sys
print(os.getcwd())
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024/multimodal_classification")
print(os.getcwd())

/content
/content/drive/MyDrive/multimodal_dataset_2024/multimodal_classification


In [ ]:
!git clone https://github.com/mlfoundations/open_clip.git

Cloning into 'open_clip'...
remote: Enumerating objects: 3543, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 3543 (delta 11), reused 9 (delta 9), pack-reused 3521 (from 2)
Receiving objects: 100% (3543/3543), 15.32 MiB | 3.71 MiB/s, done.
Resolving deltas: 100% (2145/2145), done.
Updating files: 100% (214/214), done.


In [ ]:
!pip install img2dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━

In [ ]:
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024/mscoco")
!img2dataset --url_list mscoco.parquet --input_format "parquet"\
         --url_col "URL" --caption_col "TEXT" --output_format webdataset\
           --output_folder mscoco_webdataset --processes_count 16 --thread_count 64 --image_size 256
#!unzip MMHS150K.zip -d MMHS150K
#!wget https://huggingface.co/datasets/ChristophSchuhmann/MS_COCO_2017_URL_TEXT/resolve/main/mscoco.parquet

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Starting the downloading of this file
Sharding file number 1 of 1 called /content/drive/MyDrive/multimodal_dataset_2024/mscoco/mscoco.parquet
0it [00:00, ?it/s]File sharded in 60 shards
/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip

In [ ]:
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K")






########################
import os
import json
import csv

def majority_vote(labels):
    """
    Determine the final label using majority voting.

    Args:
        labels (list): List of three numeric labels (e.g., [0, 1, 1]).

    Returns:
        int: Final label based on majority voting or the hate-class rule.
    """
    # Count occurrences of each label
    label_counts = {}
    for label in labels:
        label_counts[label] = label_counts.get(label, 0) + 1

    # Find the most common label(s)
    sorted_counts = sorted(label_counts.items(), key=lambda x: (-x[1], x[0]))
    most_common = sorted_counts[0]

    # Check for majority
    if most_common[1] >= 2:
        return most_common[0]  # Majority class

    # If all three are different, prioritize hate classes (1 to 5)
    hate_classes = [1, 2, 3, 4, 5]
    for label in labels:
        if label in hate_classes:
            return label  # Return the first hate class encountered

    # Default to "Not Hate" (0) if no hate class is present
    return 0

import os
import json
import re

# Paths
dataset_path = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K"
images_path = os.path.join(dataset_path, "img_resized")
annotations_path = os.path.join(dataset_path, "MMHS150K_GT.json")
output_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data"

train_id_split = f"{dataset_path}/splits/val_ids.txt"
with open(train_id_split, 'r') as file:
    lines = file.readlines()
train_ids = set([line.strip() for line in lines])  # Remove newline characters

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Load annotations
with open(annotations_path, "r") as f:
    annotations = json.load(f)

# Class names
class_names = ["NotHate", "Racist", "Sexist", "Homophobe", "Religion", "OtherHate"]

# Prepare JSON files for each class
json_files = {class_id: [] for class_id in range(len(class_names))}
cnt, not_found, not_train = 0, 0, 0

# Write samples to the corresponding class JSON list
for tweet_id, metadata in annotations.items():
    print(f"{cnt}")
    cnt += 1
    image_path = os.path.join(images_path, f"{tweet_id}.jpg")
    check_exist=False # to avoid gdrive too many requests!

    if check_exist and (not os.path.exists(image_path)):
        print(f"Warning: Image {image_path} not found, skipping.")
        not_found += 1
        continue
    if tweet_id not in train_ids:
        print(f"Image: {image_path}, {tweet_id} not in train_ids, skipping.")
        not_train += 1
        continue

    labels = metadata["labels"]
    labels_str = ",".join(map(str, metadata["labels"]))
    label = majority_vote([int(label) for label in labels])
    caption = metadata["tweet_text"]
    caption_cleaned = re.sub(r'http[s]?://\S+', '', caption) # removes http://someurl or https://someurl with empty string

    sample = {
        "url": image_path,
        "caption": caption_cleaned,
        "label": label,
        "tweet_id": tweet_id
    }

    json_files[label].append(sample)

# Save JSON files for each class
for class_id, samples in json_files.items():
    class_name = class_names[class_id]
    output_path = os.path.join(output_dir, f"class_{class_id}_{class_name}.json")
    with open(output_path, "w") as json_file:
        json.dump(samples, json_file, indent=4)

print(f"Metadata JSON files created for each class.")

Streaming output truncated to the last 5000 lines.
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1046515947153051651.jpg, 1046515947153051651 not in train_ids, skipping.
147304
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1107732673508843522.jpg, 1107732673508843522 not in train_ids, skipping.
147305
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1109801982536097792.jpg, 1109801982536097792 not in train_ids, skipping.
147306
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1113503840798515200.jpg, 1113503840798515200 not in train_ids, skipping.
147307
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1114515080362758152.jpg, 1114515080362758152 not in train_ids, skipping.
147308
Image: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K/img_resized/1055879376770748416.jpg, 1055879376770748416 not in train_ids, skipping.
147309
Image: /content/dri

In [ ]:
!pip install pillow

In [ ]:
import os
import json
import tarfile
from math import ceil
from PIL import Image
import cv2
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import threading
from queue import Queue
import multiprocessing

counter = multiprocessing.Value('i', 0)  # 'i' represents an integer





def resize_with_border(image_path, dest_path, target_size=(256, 256), border_mode=cv2.BORDER_CONSTANT, border_value=(255, 255, 255)):
    """
    Resize an image while maintaining its aspect ratio and adding borders to fit the target size.

    Args:
        image_path (str): Path to the input image.
        dest_path (str): Path to save the resized image.
        target_size (tuple): Desired output size (width, height).
        border_mode (int): OpenCV border mode (e.g., cv2.BORDER_CONSTANT or cv2.BORDER_REFLECT).
        border_value (tuple): Color for the border (used if border_mode is cv2.BORDER_CONSTANT).

    Returns:
        None
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Get original dimensions
    original_height, original_width = image.shape[:2]

    # Calculate aspect ratio of the target size and the image
    target_width, target_height = target_size
    target_aspect = target_width / target_height
    original_aspect = original_width / original_height

    # Resize while maintaining aspect ratio
    if original_aspect > target_aspect:
        # Image is wider than target, fit to width
        new_width = target_width
        new_height = int(target_width / original_aspect)
    else:
        # Image is taller than target, fit to height
        new_height = target_height
        new_width = int(target_height * original_aspect)

    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # Calculate border sizes
    top = (target_height - new_height) // 2
    bottom = target_height - new_height - top
    left = (target_width - new_width) // 2
    right = target_width - new_width - left

    # Add borders
    bordered_image = cv2.copyMakeBorder(
        resized_image,
        top,
        bottom,
        left,
        right,
        border_mode,
        value=border_value
    )

    # Save the final image
    cv2.imwrite(dest_path, bordered_image)
    #print(f"Resized image saved to {dest_path}")

def process_file(obj, image_size, queue):
    """
    Processes a single file: resizes the image, and adds the image, caption, and metadata to the tar file.
    """
    with counter.get_lock():  # Acquire the lock
        counter.value += 1  # Increment the counter
        print(f"{counter.value}")
    image_path = obj["url"]  # Path to the image
    caption = obj["caption"]
    label = obj["label"]
    tweet_id = obj["tweet_id"]

    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found, skipping.")
        return

    try:
        # Prepare filenames
        image_name = os.path.basename(image_path).split(".")[0]
        img_resized_path = f"{image_name}_temp.jpg"
        caption_file = f"{image_name}.txt"
        metadata_file = f"{image_name}.json"

        resize_with_border(
            image_path=image_path,
            dest_path=img_resized_path,
            target_size=image_size,
            border_mode=cv2.BORDER_CONSTANT
        )


        # Create and add caption.txt
        caption_file = f"{image_name}.txt"
        with open(caption_file, "w") as caption_f:
            caption_f.write(caption)


        # Create and add metadata.json
        metadata_file = f"{image_name}.json"
        metadata = {
            "caption": caption,
            "label": label,
            "tweet_id": tweet_id,
        }
        with open(metadata_file, "w") as metadata_f:
            json.dump(metadata, metadata_f)

        # Add processed files to the queue
        queue.put((img_resized_path, f"{image_name}.jpg"))
        queue.put((caption_file, caption_file))
        queue.put((metadata_file, metadata_file))

    except Exception as e:
        print(f"Error processing file {image_path}: {e}")

def write_to_tar(queue, tar_path):
    """
    Write processed files to the tar file from the queue.
    """
    with tarfile.open(tar_path, "w") as tar:
        while True:
            item = queue.get()
            if item is None:  # Sentinel to stop the thread
                break
            local_path, arcname = item
            tar.add(local_path, arcname=arcname)
            os.remove(local_path)  # Cleanup temporary file
            queue.task_done()


# Input paths
dataset_dir = "/path/to/dataset"  # Root directory where images are stored
json_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards"  # Directory containing class-specific JSON files
output_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset"  # Directory to store shard folders
shard_size = 10000  # Maximum number of images per .tar file
image_size = (256, 256)                 # Resize image dimensions (width, height)
max_workers = 64

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate over each class JSON file
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue

    # Parse class information from JSON file name
    class_name = os.path.splitext(json_file)[0]  # Example: class_0_NotHate.json -> class_0_NotHate
    class_shard_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_shard_dir, exist_ok=True)

    # Load the JSON data
    with open(os.path.join(json_dir, json_file), "r") as f:
        data = json.load(f)

    # Total number of shards needed
    total_shards = ceil(len(data) / shard_size)

    # Create shards
    for shard_idx in range(total_shards):
        shard_path = os.path.join(class_shard_dir, f"{str(shard_idx + 1).zfill(5)}.tar")
        print(f"Creating shard: {shard_path}")

        with tarfile.open(shard_path, "w") as tar:
            lock = threading.Lock()
            start_idx = shard_idx * shard_size
            end_idx = min((shard_idx + 1) * shard_size, len(data))
            temp_cnt = start_idx
            shard_data = data[start_idx:end_idx]

            print(f"Processing shard {shard_idx}: {start_idx} to {end_idx}...")

            # Thread-safe queue for storing processed files
            queue = Queue()

            # Start tar-writing thread
            tar_writer = threading.Thread(target=write_to_tar, args=(queue, shard_path))
            tar_writer.start()
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                executor.map(lambda obj: process_file(obj, image_size, queue), shard_data)
                '''
                futures = [
                    executor.submit(process_file, obj, tar, image_size)
                    for obj in shard_data
                ]
                for future in futures:
                    future.result()  # Wait for all threads to complete
                '''




            queue.put(None)
            tar_writer.join()
            print(f"Shard {shard_idx} completed and saved to {shard_path}.")






        print(f"Shard {shard_idx + 1}/{total_shards} for {class_name} created.")

print("All shards created successfully!")


Streaming output truncated to the last 5000 lines.
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131

Exception in thread Thread-12 (write_to_tar):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-e93a41d665ab>", line 139, in write_to_tar
  File "/usr/lib/python3.10/tarfile.py", line 2202, in add
    self.addfile(tarinfo, f)
  File "/usr/lib/python3.10/tarfile.py", line 2230, in addfile
    copyfileobj(fileobj, self.fileobj, tarinfo.size, bufsize=bufsize)
  File "/usr/lib/python3.10/tarfile.py", line 260, in copyfileobj
    raise exception("unexpected end of data")
OSError: unexpected end of data


Streaming output truncated to the last 5000 lines.
13797
13798
13799
13800
13801
13802
13803
13804
13805
13806
13807
13808
13809
13810
13811
13812
13813
13814
13815
13816
13817
13818
13819
13820
13821
13822
13823
13824
13825
13826
13827
13828
13829
13830
13831
13832
13833
13834
13835
13836
13837
13838
13839
13840
13841
13842
13843
13844
13845
13846
13847
13848
13849
13850
13851
13852
13853
13854
13855
13856
13857
13858
13859
13860
13861
13862
13863
13864
13865
13866
13867
13868
13869
13870
13871
13872
13873
13874
13875
13876
13877
13878
13879
13880
13881
13882
13883
13884
13885
13886
13887
13888
13889
13890
13891
13892
13893
13894
13895
13896
13897
13898
13899
13900
13901
13902
13903
13904
13905
13906
13907
13908
13909
13910
13911
13912
13913
13914
13915
13916
13917
13918
13919
13920
13921
13922
13923
13924
13925
13926
13927
13928
13929
13930
13931
13932
13933
13934
13935
13936
13937
13938
13939
13940
13941
13942
13943
13944
13945
13946
13947
13948
13949
13950
13951
13952
13953
13954
1

Exception in thread Thread-15 (write_to_tar):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-e503e36b4c02>", line 144, in write_to_tar
  File "/usr/lib/python3.10/tarfile.py", line 2186, in add
    tarinfo = self.gettarinfo(name, arcname)
  File "/usr/lib/python3.10/tarfile.py", line 2059, in gettarinfo
    statres = os.lstat(name)
FileNotFoundError: [Errno 2] No such file or directory: '1115089220698349568_temp.jpg'


Streaming output truncated to the last 5000 lines.
150791
150792
150793
150794
150795
150796
150797
150798
150799
150800
150801
150802
150803
150804
150805
150806
150807
150808
150809
150810
150811
150812
150813
150814
150815
150816
150817
150818
150819
150820
150821
150822
150823
150824
150825
150826
150827
150828
150829
150830
150831
150832
150833
150834
150835
150836
150837
150838
150839
150840
150841
150842
150843
150844
150845
150846
150847
150848
150849
150850
150851
150852
150853
150854
150855
150856
150857
150858
150859
150860
150861
150862
150863
150864
150865
150866
150867
150868
150869
150870
150871
150872
150873
150874
150875
150876
150877
150878
150879
150880
150881
150882
150883
150884
150885
150886
150887
150888
150889
150890
150891
150892
150893
150894
150895
150896
150897
150898
150899
150900
150901
150902
150903
150904
150905
150906
150907
150908
150909
150910
150911
150912
150913
150914
150915
150916
150917
150918
150919
150920
150921
150922
150923
150924
150925
1509

In [ ]:
import os
import json
import tarfile
from math import ceil
from PIL import Image
import cv2
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import threading
from queue import Queue
import multiprocessing

counter = multiprocessing.Value('i', 0)  # 'i' represents an integer

print("hey1")
# Input paths
dataset_dir = "/path/to/dataset"  # Root directory where images are stored
json_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards"  # Directory containing class-specific JSON files
output_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset"  # Directory to store shard folders
shard_size = 10000  # Maximum number of images per .tar file
image_size = (256, 256)                 # Resize image dimensions (width, height)
max_workers = 1 #96

# Ensure output directory exists
import shutil
# if os.path.exists(output_dir):
#   shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

os.chdir(output_dir)


def resize_with_border(image_path, dest_path, target_size=(256, 256), border_mode=cv2.BORDER_CONSTANT, border_value=(255, 255, 255)):
    """
    Resize an image while maintaining its aspect ratio and adding borders to fit the target size.

    Args:
        image_path (str): Path to the input image.
        dest_path (str): Path to save the resized image.
        target_size (tuple): Desired output size (width, height).
        border_mode (int): OpenCV border mode (e.g., cv2.BORDER_CONSTANT or cv2.BORDER_REFLECT).
        border_value (tuple): Color for the border (used if border_mode is cv2.BORDER_CONSTANT).

    Returns:
        None
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Get original dimensions
    original_height, original_width = image.shape[:2]

    # Calculate aspect ratio of the target size and the image
    target_width, target_height = target_size
    target_aspect = target_width / target_height
    original_aspect = original_width / original_height

    # Resize while maintaining aspect ratio
    if original_aspect > target_aspect:
        # Image is wider than target, fit to width
        new_width = target_width
        new_height = int(target_width / original_aspect)
    else:
        # Image is taller than target, fit to height
        new_height = target_height
        new_width = int(target_height * original_aspect)

    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # Calculate border sizes
    top = (target_height - new_height) // 2
    bottom = target_height - new_height - top
    left = (target_width - new_width) // 2
    right = target_width - new_width - left

    # Add borders
    bordered_image = cv2.copyMakeBorder(
        resized_image,
        top,
        bottom,
        left,
        right,
        border_mode,
        value=border_value
    )

    # Save the final image
    cv2.imwrite(dest_path, bordered_image)
    #print(f"Resized image saved to {dest_path}")

def process_file(obj, image_size, shard_path):
    """
    Processes a single file: resizes the image, and adds the image, caption, and metadata to the tar file.
    """
    with counter.get_lock():  # Acquire the lock
        counter.value += 1  # Increment the counter
        print(f"{counter.value}")
    image_path = obj["url"]  # Path to the image
    caption = obj["caption"]
    label = obj["label"]
    tweet_id = obj["tweet_id"]

    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found, skipping.")
        return

    try:
        # Prepare filenames
        image_name = os.path.basename(image_path).split(".")[0]
        img_resized_path = f"{image_name}_temp.jpg"
        caption_file = f"{image_name}.txt"
        metadata_file = f"{image_name}.json"

        resize_with_border(
            image_path=image_path,
            dest_path=img_resized_path,
            target_size=image_size,
            border_mode=cv2.BORDER_CONSTANT
        )


        # Create and add caption.txt
        caption_file = f"{image_name}.txt"
        with open(caption_file, "w") as caption_f:
            caption_f.write(caption)


        # Create and add metadata.json
        metadata_file = f"{image_name}.json"
        metadata = {
            "caption": caption,
            "label": label,
            "tweet_id": tweet_id,
        }
        with open(metadata_file, "w") as metadata_f:
            json.dump(metadata, metadata_f)

        # Add processed files to the queue
        queue = []
        queue.append((img_resized_path, f"{image_name}.jpg"))
        queue.append((caption_file, caption_file))
        queue.append((metadata_file, metadata_file))

        for (local_path, arcname) in queue:
          os.system(f"mv {local_path} {shard_path}/{arcname}")
          if os.path.exists(local_path):
            os.remove(local_path)  # Cleanup temporary file

    except Exception as e:
        print(f"Error processing file {image_path}: {e}")




class_names_task = ["Racist", "Sexist", "Homophobe", "Religion", "OtherHate"] #"NotHate"


# Iterate over each class JSON file
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue
    print("hey2")

    # Parse class information from JSON file name
    class_name = os.path.splitext(json_file)[0]  # Example: class_0_NotHate.json -> class_0_NotHate


    if not (class_name.split("_")[-1] in class_names_task):
      continue

    class_shard_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_shard_dir, exist_ok=True)

    # Load the JSON data
    with open(os.path.join(json_dir, json_file), "r") as f:
        data = json.load(f)

    # Total number of shards needed
    total_shards = ceil(len(data) / shard_size)
    print("hey3")

    # Create shards
    for shard_idx in range(total_shards):
        shard_path = os.path.join(class_shard_dir, f"{str(shard_idx + 1).zfill(5)}")
        # if not ("class_0_NotHate/00011" in shard_path):
        #   print(f"skipping, {shard_path}")
        #   continue

        os.makedirs(shard_path, exist_ok=True)


        print(f"Creating shard: {shard_path}")



        lock = threading.Lock()
        start_idx = shard_idx * shard_size
        end_idx = min((shard_idx + 1) * shard_size, len(data))
        temp_cnt = start_idx
        shard_data = data[start_idx:end_idx]

        print(f"Processing shard {shard_idx}: {start_idx} to {end_idx}...")

        # Thread-safe queue for storing processed files

        # Start tar-writing thread
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            executor.map(lambda obj: process_file(obj, image_size, shard_path), shard_data)
            '''
            futures = [
                executor.submit(process_file, obj, tar, image_size)
                for obj in shard_data
            ]
            for future in futures:
                future.result()  # Wait for all threads to complete
            '''





        print(f"Shard {shard_idx} completed and saved to {shard_path}.")






        print(f"Shard {shard_idx + 1}/{total_shards} for {class_name} created.")

print("All shards created successfully!")


hey1
hey2
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset/class_1_Racist/00001
Processing shard 0: 0 to 10000...
1
2

3

4
5
6
7
8
9
10
11
12

13
14

15
16
17


KeyboardInterrupt: 

In [ ]:
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards")



!img2dataset --url_list class_4_Religion_temp.json\
            --output_folder mmhs150k_webdataset\
            --input_format "json"\
            --output_format webdataset\
            --url_col "url"\
            --caption_col "caption"\
            --save_additional_columns "label,tweet_id"\
            --processes_count 16 --thread_count 64 --image_size 256 \
            --number_sample_per_shard 10000 \

 ##  --extra_metadata_columns "label,tweet_id" --save_additional_columns_in_json True\


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Traceback (most recent call last):
  File "/usr/local/bin/img2dataset", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/img2dataset/main.py", line 276, in main
    fire.Fire(download)
  File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 475, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 691, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/usr/local/lib/pyt

In [ ]:
import pandas as pd
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024")
#!unzip fakeedit/all_comments.tsv.zip -d fakeedit/all_comments/
file_path = "fakeedit/multimodal_only_samples/multimodal_validate.tsv"


df = pd.read_csv(file_path, sep='\t')  # Read as TSV (tab-separated)

# Add the new column with a constant value 'fakeedit'
df['dataset'] = 'fakeedit'

# Get unique values in the "6_way_label" column
unique_labels = df["6_way_label"].unique()

# Create a separate TSV file for each label
for label in unique_labels:
    # Filter the dataframe for the current label
    df_label = df[df["6_way_label"] == label]

    # Create an output file path for this label
    new_file_path = f"fakeedit/validation_data/multimodal_validate_label_{label}.tsv"  # Example: output_label_0.tsv

    # Save the filtered dataframe as a new TSV file
    df_label.to_csv(new_file_path, sep='\t', index=False)
    print(f"Saved {new_file_path} with {len(df_label)} rows.")

Saved fakeedit/validation_data/multimodal_validate_label_0.tsv with 23320 rows.
Saved fakeedit/validation_data/multimodal_validate_label_4.tsv with 17810 rows.
Saved fakeedit/validation_data/multimodal_validate_label_2.tsv with 11277 rows.
Saved fakeedit/validation_data/multimodal_validate_label_1.tsv with 3521 rows.
Saved fakeedit/validation_data/multimodal_validate_label_5.tsv with 2176 rows.
Saved fakeedit/validation_data/multimodal_validate_label_3.tsv with 1238 rows.


In [ ]:
df.head()

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label,dataset
0,singingdart7854,my xbox controller says hi,1.567436e+09,i.redd.it,True,cypw96,https://preview.redd.it/l0ga0tug17k31.jpg?widt...,NaN,4.0,25,mildlyinteresting,My Xbox controller says hi,0.72,1,0,0,fakeedit
1,mandal0re,new image from the mandalorian,1.567745e+09,i.imgur.com,True,d0bzlq,https://external-preview.redd.it/VX7bXDu9Gl8UZ...,NaN,5.0,21,photoshopbattles,PsBattle: New image from The Mandalorian,0.92,1,0,0,fakeedit
2,HE_WHO_DRUELS,say hello to my little friend,1.461468e+09,NaN,True,d2ezoob,http://i.imgur.com/F1Zbl3D.jpg,4g6bp9,NaN,10,psbattle_artwork,Say hello to my little friend!,NaN,0,2,4,fakeedit
3,eNaRDe,watch your step little one,1.408047e+09,NaN,True,cjqctpw,http://i.imgur.com/KRyMjn1.jpg,2diyh3,NaN,1,psbattle_artwork,Watch your step little one,NaN,0,2,4,fakeedit
4,Thebubster2001,this tree i found with a solo cup on it,1.558186e+09,i.redd.it,True,bq3yuk,https://preview.redd.it/bxp58zf01zy21.jpg?widt...,NaN,8.0,6,mildlyinteresting,This tree I found with a solo cup on it,0.62,1,0,0,fakeedit


In [ ]:
!pip install img2dataset
!pip install  albumentations==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.24
    Uninstalling albumentations-1.4.24:
      Successfully uninstalled albumentations-1.4.24


In [ ]:
os.chdir("/content/drive/MyDrive/multimodal_dataset_2024/fakeedit")

!img2dataset --url_list ./multimodal_train_label_4.tsv --input_format "tsv"\
         --url_col "image_url" --caption_col "clean_title" --output_format webdataset\
         --save_additional_columns '["6_way_label","dataset"]'\
         --output_folder ./temp_webdataset/label_4 --processes_count 16 --thread_count 64 --image_size 256\
         --retries 2 --timeout 10 --number_sample_per_shard 1000
#"6_way_label" "dataset"

Starting the downloading of this file
Sharding file number 1 of 1 called /content/drive/MyDrive/multimodal_dataset_2024/fakeedit/multimodal_train_label_4.tsv
0it [00:00, ?it/s]File sharded in 168 shards
13it [00:14,  1.23it/s]worker  - success: 0.023 - failed to download: 0.974 - failed to resize: 0.003 - images per sec: 182 - count: 1000
total   - success: 0.023 - failed to download: 0.974 - failed to resize: 0.003 - images per sec: 182 - count: 1000
worker  - success: 0.015 - failed to download: 0.984 - failed to resize: 0.001 - images per sec: 177 - count: 1000
total   - success: 0.019 - failed to download: 0.979 - failed to resize: 0.002 - images per sec: 354 - count: 2000
worker  - success: 0.018 - failed to download: 0.979 - failed to resize: 0.003 - images per sec: 254 - count: 1000
total   - success: 0.019 - failed to download: 0.979 - failed to resize: 0.002 - images per sec: 314 - count: 3000
worker  - success: 0.014 - failed to download: 0.985 - failed to resize: 0.001 - ima

In [ ]:
df['6_way_label'].unique()

array([0, 4, 2, 1, 5, 3])

In [ ]:


import argparse
import json
import os
from multiprocessing import Process
import webdataset as wds
from more_itertools import divide
from tqdm import tqdm




def group_files_by_id(folder_path):
    """ Groups files by ID (common prefix before extension). """
    grouped_files = {}
    for file_name in os.listdir(folder_path):
        if not file_name.startswith('.'):  # Ignore hidden files
            file_id, ext = os.path.splitext(file_name)
            grouped_files.setdefault(file_id, []).append(file_name)
    return grouped_files


def convert(file_groups, opts, pid):
    """ Convert grouped files into WebDataset format. """
    shard_path = f"{opts.output_dir}/0%02d{opts.shard_number:02d}.tar"
    #shard_path = f"{opts.output_dir}/{pid:02d}%06d.tar"
    print(f"Creating shard: {shard_path}")
    with wds.ShardWriter(shard_path, maxcount=opts.maxcount_per_shard) as sink:
        for file_id, files in tqdm(file_groups.items(), desc=f"Process {pid}", leave=False, position=pid):
            sample = {"__key__": file_id}
            for file_name in files:
                ext = os.path.splitext(file_name)[1].lower()
                full_path = os.path.join(opts.input_dir, file_name)
                if ext == ".jpg":
                    with open(full_path, "rb") as f:
                        jpg_data = f.read()

                elif ext == ".txt":
                    with open(full_path, "r", encoding="utf-8") as f:
                        txt_data = f.read()

                elif ext == ".json":
                    with open(full_path, "r", encoding="utf-8") as f:
                        json_data = f.read()

            sample["jpg"] = jpg_data
            sample["txt"] = txt_data
            sample["json"] = json_data
            sink.write(sample)


def convert_all(file_groups_all, opts):
    """ Convert all grouped files into WebDataset shards using multiprocessing. """
    procs = []
    for pid, file_groups in enumerate(divide(min(opts.cores, len(file_groups_all)), file_groups_all)):
        p = Process(target=convert, args=(dict(file_groups), opts, pid))
        p.start()
        procs.append(p)

    for p in procs:
        p.join()


def main(opts):
    # Group files by their ID
    file_groups = group_files_by_id(opts.input_dir)
    grouped_items = list(file_groups.items())
    convert_all(grouped_items, opts)



folder_number = "00010"
folder_path = f"/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset/class_0_NotHate"
os.chdir(folder_path)

class CustomArgs:
    pass

args = CustomArgs()




args.input_dir = f"{folder_path}/{folder_number}"
args.output_dir = f"{folder_path}"
args.cores = 1 #os.cpu_count()
args.maxcount_per_shard = 10000
args.shard_number = int(folder_number)
main(args)



Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset/class_0_NotHate/0%02d10.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/MMHS150K_webdataset/class_0_NotHate/00010.tar 0 0.0 GB 0


In [18]:
import os
import json
import tarfile
from math import ceil
from PIL import Image
import cv2
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import threading
from queue import Queue
import multiprocessing

import argparse
import json
import os
import zipfile
from multiprocessing import Process

import webdataset as wds
from  img2dataset.resizer import Resizer
from more_itertools import divide
from tqdm import tqdm


def group_files_by_id(folder_path):
    """ Groups files by ID (common prefix before extension). """
    grouped_files = {}
    for file_name in os.listdir(folder_path):
        if not file_name.startswith('.'):  # Ignore hidden files
            file_id, ext = os.path.splitext(file_name)
            grouped_files.setdefault(file_id, []).append(file_name)
    return grouped_files


def convert(file_groups, resizer, opts, pid):
    """ Convert grouped files into WebDataset format. """
    shard_path = f"{opts.output_dir}/0%02d{opts.shard_number:02d}.tar"
    print(f"Creating shard: {shard_path}")
    with wds.ShardWriter(shard_path, maxcount=opts.maxcount_per_shard) as sink:
        for file_id, obj in tqdm(enumerate(file_groups), total=len(file_groups), desc=f"Process {pid}", leave=False, position=pid):
            image_path = obj["url"]  # Path to the image
            caption = obj["caption"]
            label = obj["label"]
            tweet_id = obj["tweet_id"]
            image_name = os.path.basename(image_path).split(".")[0]
            sample = {"__key__": image_name}

            metadata = {
              "caption": caption,
              "label": label,
              "tweet_id": tweet_id,
            }
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_path} not found, skipping.")
                continue

            with open(image_path, "rb") as f:
                (img_str, width, height, original_width, original_height, error_message,) = resizer(f)
                #jpg_data = f.read()




            sample["jpg"] = img_str
            sample["txt"] = caption
            sample["json"] = metadata
            sink.write(sample)


def convert_all(file_groups_all, opts):
    """ Convert all grouped files into WebDataset shards using multiprocessing. """
    resizer = Resizer(image_size=image_size[0], resize_mode="border", resize_only_if_bigger=False, )

    procs = []
    for pid, file_groups in enumerate(divide(min(opts.cores, len(file_groups_all)), file_groups_all)):
        p = Process(target=convert, args=(list(file_groups), resizer, opts, pid))
        p.start()
        procs.append(p)

    for p in procs:
        p.join()





counter = multiprocessing.Value('i', 0)  # 'i' represents an integer

print("hey1")
# Input paths
dataset_dir = "/path/to/dataset"  # Root directory where images are stored
json_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data"  # Directory containing class-specific JSON files
output_dir = "/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset"  # Directory to store shard folders
shard_size = 1000  # Maximum number of images per .tar file
image_size = (256, 256)                 # Resize image dimensions (width, height)
max_workers = 1 #96

# Ensure output directory exists
import shutil
# if os.path.exists(output_dir):
#   shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

os.chdir(output_dir)


def resize_with_border(image_path, dest_path, target_size=(256, 256), border_mode=cv2.BORDER_CONSTANT, border_value=(255, 255, 255)):
    """
    Resize an image while maintaining its aspect ratio and adding borders to fit the target size.

    Args:
        image_path (str): Path to the input image.
        dest_path (str): Path to save the resized image.
        target_size (tuple): Desired output size (width, height).
        border_mode (int): OpenCV border mode (e.g., cv2.BORDER_CONSTANT or cv2.BORDER_REFLECT).
        border_value (tuple): Color for the border (used if border_mode is cv2.BORDER_CONSTANT).

    Returns:
        None
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Get original dimensions
    original_height, original_width = image.shape[:2]

    # Calculate aspect ratio of the target size and the image
    target_width, target_height = target_size
    target_aspect = target_width / target_height
    original_aspect = original_width / original_height

    # Resize while maintaining aspect ratio
    if original_aspect > target_aspect:
        # Image is wider than target, fit to width
        new_width = target_width
        new_height = int(target_width / original_aspect)
    else:
        # Image is taller than target, fit to height
        new_height = target_height
        new_width = int(target_height * original_aspect)

    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    # Calculate border sizes
    top = (target_height - new_height) // 2
    bottom = target_height - new_height - top
    left = (target_width - new_width) // 2
    right = target_width - new_width - left

    # Add borders
    bordered_image = cv2.copyMakeBorder(
        resized_image,
        top,
        bottom,
        left,
        right,
        border_mode,
        value=border_value
    )

    # Save the final image
    cv2.imwrite(dest_path, bordered_image)
    #print(f"Resized image saved to {dest_path}")






class_names_task = ["NotHate", "Racist", "Sexist", "Homophobe", "Religion", "OtherHate"] #""

class CustomArgs:
    pass

args = CustomArgs()
args.output_dir = ""
args.cores = 1 #os.cpu_count()
args.maxcount_per_shard = 1000
args.shard_number = None

# Iterate over each class JSON file
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue
    print("hey2")

    # Parse class information from JSON file name
    class_name = os.path.splitext(json_file)[0]  # Example: class_0_NotHate.json -> class_0_NotHate


    if not (class_name.split("_")[-1] in class_names_task):
      continue

    class_shard_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_shard_dir, exist_ok=True)

    # Load the JSON data
    with open(os.path.join(json_dir, json_file), "r") as f:
        data = json.load(f)

    # Total number of shards needed
    total_shards = ceil(len(data) / shard_size)
    print("hey3")

    # Create shards
    for shard_idx in range(total_shards):
        #shard_path = os.path.join(class_shard_dir, f"{str(shard_idx + 1).zfill(5)}")
        # if not ("class_0_NotHate/00011" in shard_path):
        #   print(f"skipping, {shard_path}")
        #   continue
        args.shard_number = shard_idx + 1
        args.output_dir = class_shard_dir





        start_idx = shard_idx * shard_size
        end_idx = min((shard_idx + 1) * shard_size, len(data))
        temp_cnt = start_idx
        shard_data = data[start_idx:end_idx]
        convert_all(shard_data, args)

        print(f"Processing shard {shard_idx}: {start_idx} to {end_idx}...")

        print(f"Shard {shard_idx + 1}/{total_shards} for {class_name} created.")



print("All shards created successfully!")

hey1
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/0%02d01.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/00001.tar 0 0.0 GB 0


Processing shard 0: 0 to 1000...
Shard 1/3 for class_0_NotHate created.
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/0%02d02.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/00002.tar 0 0.0 GB 0


Processing shard 1: 1000 to 2000...
Shard 2/3 for class_0_NotHate created.
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/0%02d03.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_0_NotHate/00003.tar 0 0.0 GB 0


Processing shard 2: 2000 to 2501...
Shard 3/3 for class_0_NotHate created.
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_1_Racist/0%02d01.tar/content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_1_Racist/00001.tar
# writing  0 0.0 GB 0


Processing shard 0: 0 to 1000...
Shard 1/2 for class_1_Racist created.
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_1_Racist/0%02d02.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_1_Racist/00002.tar 0 0.0 GB 0


Processing shard 1: 1000 to 1040...
Shard 2/2 for class_1_Racist created.
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_2_Sexist/0%02d01.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_2_Sexist/00001.tar 0 0.0 GB 0


Processing shard 0: 0 to 437...
Shard 1/1 for class_2_Sexist created.
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_3_Homophobe/0%02d01.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_3_Homophobe/00001.tar 0 0.0 GB 0


Processing shard 0: 0 to 350...
Shard 1/1 for class_3_Homophobe created.
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_4_Religion/0%02d01.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_4_Religion/00001.tar 0 0.0 GB 0


Processing shard 0: 0 to 38...
Shard 1/1 for class_4_Religion created.
hey2
hey3
Creating shard: /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_5_OtherHate/0%02d01.tar
# writing /content/drive/MyDrive/multimodal_dataset_2024/MMHS150K_shards/validation_data/MMHS150K_webdataset/class_5_OtherHate/00001.tar 0 0.0 GB 0


Processing shard 0: 0 to 634...
Shard 1/1 for class_5_OtherHate created.
All shards created successfully!
